In [1]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
pd.get_option("display.max_columns", None)

20

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                            dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                            dataset.iloc[indices.flatten()[i]]['col_1'],
                            dataset.iloc[indices.flatten()[i]]['col_2'],
                            dataset.iloc[indices.flatten()[i]]['col_3'],
#                            dataset.iloc[indices.flatten()[i]]['col_4'],
#                            dataset.iloc[indices.flatten()[i]]['col_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
#                            dataset.iloc[indices.flatten()[i]]['F_SMA_20']
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1',
                      'col_2',
                      'col_3',
#                      'col_4',
#                      'col_5',
                      'F_SMA_5',
                      'F_SMA_10',
#                      'F_SMA_20',
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [6]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'RSI', 'ATR_14',
       'SMA_5', 'SMA_10', 'SMA_20', 'O-H', 'O-L', 'O-C', 'H-L', 'H-C', 'L-C',
       'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'F_SMA_5',
       'F_SMA_10', 'F_SMA_20'],
      dtype='object')

In [7]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,L-C,Direction,col_1,col_2,col_3,col_4,col_5,F_SMA_5,F_SMA_10,F_SMA_20
0,2015-09-03,01:00:00,1,246,3,Thursday,2015-09-03T01:00:00.000000000Z,2065,1.12134,1.12245,...,-0.00060,1,-0.00052,0.00111,0.00060,0.00131,0.00085,-0.00084,-0.00364,-0.00274
1,2015-09-03,05:00:00,5,246,3,Thursday,2015-09-03T05:00:00.000000000Z,5490,1.12190,1.12434,...,-0.00263,1,-0.00215,0.00244,0.00263,-0.00052,0.00131,0.00135,-0.00115,-0.00065
2,2015-09-03,09:00:00,9,246,3,Thursday,2015-09-03T09:00:00.000000000Z,13522,1.12408,1.12417,...,-0.00090,0,0.01215,0.00009,0.00090,-0.00215,-0.00052,-0.00847,-0.01127,-0.01207
3,2015-09-03,13:00:00,13,246,3,Thursday,2015-09-03T13:00:00.000000000Z,13816,1.11193,1.11515,...,-0.00298,0,0.00021,0.00322,0.00298,0.01215,-0.00215,-0.00648,-0.00998,-0.01188
4,2015-09-03,17:00:00,17,246,3,Thursday,2015-09-03T17:00:00.000000000Z,6221,1.11170,1.11398,...,-0.00106,1,-0.00066,0.00228,0.00106,0.00021,0.01215,-0.00404,-0.00774,-0.01074


In [8]:
Test_Candle = [7471, 2266, 3273, 6415, 7960, 8958, 8193,  244, 3656, 8882, 6064,
               6260, 4380,  774, 1501, 7142, 3587, 8581, 5822, 5502, 8422, 1648,
               6006, 1563, 1491, 6141, 1638, 4923,  428, 4906, 2846, 1234, 3430,
               1707,  980, 2533, 2208, 1339, 6643, 4097, 9092, 8089, 9445, 6349,
               6855, 5529, 2842, 2795, 3356,  262, 5183, 4298, 8963, 6222, 2240,
               6310, 1554, 7529, 6482, 8065, 7845, 9524, 5522, 8154, 3884, 5688,
                863, 3261, 3943, 3401, 5029, 4170, 8752, 3245,  301, 6612, 3499,
               7978, 8354, 9244, 8695, 4272, 4247, 1628, 9396, 9590, 8168, 5229,
               2668, 2110, 7143, 7732, 3897, 8303, 5988, 2772, 8801, 3707, 6302,
               7447, 3309, 3199, 3335, 1189, 2142, 2756, 7595, 4600, 2595, 6831,
                655, 7353, 4491, 6020, 9255, 7435,  868, 5074, 3702, 2278, 7873,
                475,  836, 8555, 5244, 1598, 5814, 4511, 1469, 8883, 8238,  524,
               4847, 7326, 1191, 9588, 3294, 1585, 2845, 4015, 1548, 8046, 1010,
               9373,  355, 7943, 3482, 8107, 1147, 1999, 6655, 1048, 2603, 8926,
               7721, 1590, 3686, 4402, 8360, 7518, 5232, 4399, 8664, 3456, 3147,
                942, 6253, 7924, 2363, 1027, 4560, 8572, 5079, 9052, 3441, 7304,
               7826, 7776, 4360, 5931, 2099, 4590, 2640, 4140, 3059, 3911, 8765,
               3778, 8792, 1203, 6412, 2286, 6197, 2696, 5629, 2999, 5440,   74,
               6622, 4000, 5944,   48,  772,  217, 4018, 8974, 8440, 3775, 7004,
               4374, 7039, 1440, 8062,  174, 7778, 4612, 5686, 6183, 6423, 3563,
               6261, 2152, 3332, 4262, 4997, 1794, 5624, 1320, 7175, 8661, 1603,
               7115, 8077, 8888, 5682, 2625, 4481, 4534, 4421, 1131, 6487, 1517,
               3098, 2622, 4797, 3862,  660, 7214,  619, 3663, 3539, 1211, 5557,
               4401, 2756, 2936, 2515, 9016, 2285,   70,  787, 1593, 8500, 9202,
               8128, 5684, 9282, 3448, 4169, 8343, 4713, 3941, 9199, 1694, 4260,
               4613, 7468, 5427, 2784,  227, 3893, 1674, 5548, 9164, 2495,  654,
               7394, 4604, 7069,  133, 6796, 3435, 1652, 3292, 2205, 2616,  722,
               3398, 8843, 7393, 1827, 6015, 5938, 7815, 7035, 7959, 6733,  842,
               6125, 6484, 6421, 9248,  611, 8862,  809,  157, 8134, 7905, 4301,
               2374, 8192, 4739,  197, 2170, 2320, 8045, 3306, 6548, 8517, 6808,
               7259,  837, 5341, 4865, 3465, 1554, 8248, 2832, 3535, 5308, 4307,
               6953, 6944, 3147, 7921, 6258, 9443, 7113, 4167, 4320, 9093, 4978,
                145, 7431, 7350, 2319, 5368, 1961, 2758, 4186, 5371, 7861,  488,
               3601, 5299, 3893, 7289, 6390, 4610,  355,  746, 4793, 7045, 5944,
               8136,  143, 8012, 1759, 1722, 3179, 3761, 7070, 4098, 4031, 2829,
               8715, 6354, 1403,  441, 8886, 5657, 2869, 5921, 3472, 4855, 8159,
               2200, 8893, 7603, 2763, 1215, 3029, 4545, 2040, 9332,  318, 2095,
               3168, 4358, 9568, 9300, 9474, 5213, 3059, 5967, 3935, 1521, 9181,
               5361,  656, 7668, 1663, 3716, 4580, 2257,  854, 8338,  267, 9235,
               7330, 9459, 8458, 1133, 8160,  666,  389, 1836,  544, 4169, 9482,
               6244, 8348, 5873, 9114, 6682, 5628, 3398, 9044, 8506, 6036, 5946,
                789, 5469, 9496, 6892, 9374, 6612, 4700, 6094,  959, 6513, 6752,
               8788, 6731, 3766, 8339, 2005, 8005, 2635, 7428, 6908, 1622, 5900,
               6778, 1918, 1434, 1203, 8099, 5514, 6554, 6410, 3224, 7264, 9391,
               2436, 5113, 9199, 4574, 2675, 7069, 6033, 9230, 8994, 8336, 9434,
               3593, 8810, 8933, 2303, 6275, 2316, 8207, 1181, 2413, 5096, 5793,
               2217, 8014, 7789, 2297, 2921, 8639, 2538, 3910, 3379, 1457, 4923,
                486, 2473, 5585, 2958, 7897, 2815, 7745, 2171,  950, 5241,  453,
               4073, 3858, 9005,  154, 5668, 8051, 2216, 4336, 7848, 8161, 1783,
               6369, 1562, 8845, 2313, 7245, 6188, 9545, 7012,  331, 4282, 4536,
               6345,  892,  654, 9069, 5665, 1776,  772, 8129, 7477, 1808, 9552,
                414, 6950, 8764, 3862, 5500, 6121, 8804, 7231,  491, 5518, 2554,
               2193, 5366, 6759, 4781, 2916, 5775, 7399, 8775, 6800, 7399, 7532,
               8794, 3684, 6351, 2401, 7250, 3133, 1449, 2353, 1402, 3778, 9174,
               5944, 1062, 3174,   66, 8258, 4161, 6419, 3807, 6054, 6310, 8658,
               5739, 6557, 8389, 8224, 9129, 1868, 8003,  683, 5478, 6821, 1388,
               4710, 5122, 4661, 5607,  711, 6190, 6315, 1163, 5670, 4587, 8140,
                573, 6123, 4048, 5875, 7951, 3180, 3644, 1279, 6307, 2485, 1266,
               1944, 3128, 2875,   31, 4988, 3371, 2431, 2030, 6594, 4631, 4214,
               1394, 3891, 1968,  407, 8538, 8771, 4135, 3081,  828,  316, 3094,
               8163, 7397, 2666, 4755, 9400, 1541, 7746, 1393, 2277, 7187,  486,
               6742, 1479, 6088, 1228, 4414, 5374, 6133, 7989, 6868,  175, 4379,
               3369, 6350, 4866,  231, 5975, 6176, 2932, 3486,   50, 8715, 5509,
               4703, 7956,  756, 7122, 3793, 6414, 3150, 7231, 6818, 2052, 6296,
                 72, 6174, 5705, 4266, 7722, 8818, 5906,  592, 9348, 5295, 2111,
               9043, 7598, 6433,  866, 1538, 2509,  680, 1138, 8014, 8551, 5522,
               3358, 2591, 3779, 1146,  947, 5569, 7460, 3362, 2980, 1296, 3051,
                724, 1213, 6045, 5051, 6465, 6515, 2390, 2574, 2644, 5364,  214,
               4664, 8010, 2143, 5592, 1291, 9058, 7905, 8669, 6065, 6951,  549,
                964, 8304, 8312, 4582, 2345,  104, 5652,  716, 3602,  567, 8809,
               2842, 7138, 1869, 2080, 4362, 5139, 6921, 8498, 8366, 1095, 3862,
               8945, 1906, 7806, 1502, 5095, 8962, 2841, 8360, 2645,  785, 3803,
               8553, 5764, 2354,  557, 5471, 6165, 4268,   83,  533, 8903, 2426,
               8280, 2696, 3508, 1870, 2866, 5290, 5375, 5507, 8816, 5619, 8677,
               6956, 3964, 9532, 4826, 4835, 3531, 4078, 9104, 8132, 9557, 7165,
               4675, 6628, 4636, 8349, 7057, 5056, 7445, 4670, 3623, 3529, 5595,
                248, 5183, 2757, 3888, 3290, 1242, 1578, 1857, 9019, 3610, 5374,
               9191, 9290,  142, 7179, 7691, 9365, 8279, 7821, 5239, 2292, 8170,
               7740,  113, 5782, 2468, 8868, 3831, 8934, 9326, 4290, 7706, 8948,
               5322, 6152, 5215, 7903, 9151, 6977, 8871, 8702, 5131, 5543, 8992,
               2091, 7586, 6845, 9318, 8616, 2636, 3870, 2583, 4529, 4265, 8607,
               3903,  268, 4807, 8522, 2015, 6905, 5019, 7918, 8955, 6269,  983,
               9170, 1262,  604, 6673, 9354, 1472, 2642,  423, 5688, 6769, 1269,
               1292, 2582, 3693, 2478, 7117, 9297, 1495,   54, 6796, 3169, 5127,
               3170, 3574, 4769, 5196, 3661, 7539, 7938, 5868, 4769, 7740,  725,
               4009,  900, 8803, 8538, 4168, 5689, 7117, 6368, 6003, 2887, 4182,
                686, 6072,  839, 1759,  842, 4194, 5796, 4441, 5844, 6949,   50,
               3243, 7337, 8491, 5045, 2503, 5880, 2030, 7780, 6030, 6130, 5241,
               8600, 5110, 5387, 6462, 9061,  339, 4333, 3198,  128, 3694, 6718,
               4308, 3447,  704, 6518,  962, 8240, 3296, 3626, 1643, 8239,   56,
               2626, 5726, 8516, 7531, 1643, 4006,   82, 3794, 2088, 9060
              ]

In [9]:
data.shape

(9644, 32)

In [10]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,L-C,Direction,col_1,col_2,col_3,col_4,col_5,F_SMA_5,F_SMA_10,F_SMA_20
0,2015-09-03,01:00:00,1,246,3,Thursday,2015-09-03T01:00:00.000000000Z,2065,1.12134,1.12245,...,-0.00060,1,-0.00052,0.00111,0.00060,0.00131,0.00085,-0.00084,-0.00364,-0.00274
1,2015-09-03,05:00:00,5,246,3,Thursday,2015-09-03T05:00:00.000000000Z,5490,1.12190,1.12434,...,-0.00263,1,-0.00215,0.00244,0.00263,-0.00052,0.00131,0.00135,-0.00115,-0.00065


# <font color='red'>CANDLE LOOP</font>

In [11]:
%%time

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:

    #print('candle number is = ',candle_no)
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+3]
    #print('data is = ', data)
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'
        Current_Market_Stoploss= 0
    else:
        Current_Market = 'Bearish'
        Current_Market_Stoploss= 0 
    
    data = pd.read_csv(filename)
    data = data.drop(columns=['Volume','Weekday','Date','Time','f_time','julian_date',
                              'Weekday_Name','UTC_Time','Direction',
                              'Open', 'High', 'Low', 'Close',
                              'O-H','O-L','O-C','H-L','H-C','L-C',
                              'SMA_5','SMA_10','SMA_20',                           
                              'ATR_14',
#                              'col_1',
#                              'col_2',
#                              'col_3',
                              'col_4','col_5',
#                              'F_SMA_5',
#                              'F_SMA_10',
                              'F_SMA_20',
                              'RSI',
                             ])

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+3]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY'
        else:
            Predicted_Trade = 'SELL'
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])

        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

CPU times: user 3min 22s, sys: 31.9 s, total: 3min 54s
Wall time: 3min 54s


In [12]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [13]:
result_output.head(10)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,7471.0,7.0,Bearish,0.0,50.0,BUY,97.0,SELL,64.0,BUY
1,2266.0,8.0,Bearish,0.0,51.0,SELL,66.0,BUY,99.0,SELL
2,3273.0,81.0,Bullish,0.0,67.0,BUY,60.0,BUY,8.0,SELL
3,6415.0,75.0,Bearish,0.0,63.0,BUY,91.0,BUY,22.0,SELL
4,7960.0,26.0,Bullish,0.0,42.0,BUY,14.0,BUY,72.0,SELL
5,8958.0,21.0,Bearish,0.0,54.0,BUY,67.0,BUY,8.0,BUY
6,8193.0,97.0,Bullish,0.0,100.0,SELL,94.0,BUY,94.0,SELL
7,244.0,1.0,Bullish,0.0,89.0,BUY,41.0,BUY,12.0,BUY
8,3656.0,71.0,Bullish,0.0,37.0,BUY,99.0,BUY,23.0,BUY
9,8882.0,71.0,Bullish,0.0,94.0,SELL,43.0,BUY,55.0,BUY


In [14]:
result_output.tail(10)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
990,2626.0,3.0,Bearish,0.0,98.0,SELL,69.0,BUY,0.0,BUY
991,5726.0,74.0,Bullish,0.0,95.0,SELL,27.0,SELL,79.0,BUY
992,8516.0,93.0,Bearish,0.0,62.0,BUY,97.0,SELL,0.0,SELL
993,7531.0,84.0,Bullish,0.0,19.0,SELL,57.0,SELL,99.0,BUY
994,1643.0,17.0,Bearish,0.0,100.0,SELL,85.0,SELL,42.0,SELL
995,4006.0,63.0,Bearish,0.0,62.0,SELL,31.0,BUY,56.0,BUY
996,82.0,8.0,Bullish,0.0,4.0,SELL,67.0,BUY,14.0,SELL
997,3794.0,75.0,Bullish,0.0,80.0,BUY,92.0,BUY,95.0,SELL
998,2088.0,76.0,Bullish,0.0,99.0,BUY,67.0,SELL,68.0,BUY
999,9060.0,39.0,Bullish,0.0,100.0,BUY,91.0,BUY,22.0,BUY
